In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("Midterm Project")\
    .getOrCreate()

print("Spark Session created successfully!")
spark

Spark Session created successfully!


In [4]:
from google.colab import files

uploaded = files.upload()

Saving articles.csv to articles.csv
Saving transactions-2p-42.csv to transactions-2p-42.csv


In [5]:
from pyspark.sql.functions import col, sum, when, count, to_date, year, countDistinct, lit
from pyspark.sql.types import DateType

In [15]:
#Data Quality

In [6]:
transactions_df = spark.read.csv(
    "transactions-2p-42.csv",
    header=True,
    inferSchema=True
)
articles_df = spark.read.csv(
    "articles.csv",
    header=True,
    inferSchema=True
)

In [7]:
print("--- Transactions DataFrame: Head and Schema ---")
transactions_df.show(5, truncate=False)
transactions_df.printSchema()

print("\n--- Articles DataFrame: Head and Schema ---")
articles_df.show(5, truncate=False)
articles_df.printSchema()

--- Transactions DataFrame: Head and Schema ---
+----------+----------------------------------------------------------------+----------+------------------+----------------+
|t_dat     |customer_id                                                     |article_id|price             |sales_channel_id|
+----------+----------------------------------------------------------------+----------+------------------+----------------+
|2019-07-24|192e9ad7f0c05d89e3321a3a8caa990f19cf761ad0d13794a31202b8d012939f|692721005 |0.0121864406779661|2               |
|2019-04-07|70c1ee207c64a65234a6832a42a66a6f9bae6fbf2061216cbe4461cc5afcccb0|599502013 |0.0508305084745762|2               |
|2019-05-21|e29656435a0c04ef196d7218d1ec18d800f1ff382b9442b144e47665d338f50a|737260001 |0.0254067796610169|2               |
|2019-03-29|13d1fd878959e117e6b416cba433e1b9b2867f3a423fa5dd7fb8efc1ccf500f3|717251003 |0.0169322033898305|2               |
|2019-07-05|58ddbcfa96eab2b3aaf2e05ce2148246b2f97002c50c0c3a91743d2043af6ad9|

In [11]:
all_transaction_cols = transactions_df.columns

transactions_all_null_counts = transactions_df.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in all_transaction_cols]
).collect()[0].asDict()

print("--- Null Counts for ALL transactions-2p-42.csv Columns ---")
all_zero = True
for col_name, null_count in transactions_all_null_counts.items():
    if null_count > 0:
        print(f"  {col_name}: {null_count}")
        all_zero = False

if all_zero:
    print("  All columns have ZERO null values. Data quality is excellent.")

--- Null Counts for ALL transactions-2p-42.csv Columns ---
  All columns have ZERO null values. Data quality is excellent.


In [10]:
all_article_cols = articles_df.columns

articles_all_null_counts = articles_df.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in all_article_cols]
).collect()[0].asDict()

for col_name, null_count in articles_all_null_counts.items():
    if null_count > 0:
        print(f"  {col_name}: {null_count}")

--- Null Counts for ALL Articles.csv Columns ---
  detail_desc: 416


In [12]:
#The transactional data is fully complete, while the article data has only minor incompleteness, specifically 416 null values in the detail_desc column.

In [14]:
#We have to change the type of t_dat from text to date.

In [13]:
from pyspark.sql.functions import col, to_date

transactions_clean_df = transactions_df.withColumn(
    "t_dat",
    to_date(col("t_dat"), "yyyy-MM-dd")
)

transactions_clean_df.printSchema()

root
 |-- t_dat: date (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- sales_channel_id: integer (nullable = true)



In [17]:
#Checking for Join Mismatches

In [16]:
orphan_transactions_count = transactions_clean_df.join(
    articles_df,
    on="article_id",
    how="left_anti"
).count()

total_transactions = transactions_clean_df.count()

print("--- Join Integrity Check ---")
print(f"Total transactions in sample: {total_transactions}")
print(f"Transactions without a matching article_id: {orphan_transactions_count}")
print(f"Percentage mismatch: {(orphan_transactions_count / total_transactions * 100):.4f}%")

--- Join Integrity Check ---
Total transactions in sample: 336078
Transactions without a matching article_id: 0
Percentage mismatch: 0.0000%


In [18]:
#The Left Anti Join is a filtering operation that exclusively returns the set of rows from the left DataFrame (transactions_clean_df) where the join key (article_id) is absent in the right DataFrame (articles_df)

In [ ]:
#Checking outliers for "price" column

In [19]:
transactions_df.describe("price").show()

+-------+--------------------+
|summary|               price|
+-------+--------------------+
|  count|              336078|
|   mean|0.027498519008053526|
| stddev| 0.01921244097171475|
|    min|   1.864406779661E-4|
|    max|  0.5067796610169492|
+-------+--------------------+



In [22]:
#The maximum price  is an extreme outlier, showing it's much higher than what most products sell for.
#The minimum price is positive, meaning every item sold in the sample had a correct

In [20]:
transactions_df.groupBy("sales_channel_id").count().orderBy("sales_channel_id").show()

+----------------+------+
|sales_channel_id| count|
+----------------+------+
|               1|103464|
|               2|232614|
+----------------+------+



In [23]:
#We can conclude that 69.22% of all transactions originate from Sales Channel 2

In [25]:
#What are the product trend changes through (3 points)
   # a. seasons of the year
   # b. the years

In [24]:
from pyspark.sql.functions import col, year, month, sum, lit, when

In [38]:
# a. seasons of the year

In [30]:
#Doing inner join

In [28]:
master_df = transactions_clean_df.join(
    articles_df.select("article_id", "product_group_name", "index_name"),
    on="article_id",
    how="inner"
).withColumn("purchase_year", year(col("t_dat")))\
 .withColumn("purchase_month", month(col("t_dat")))


In [29]:
master_df

DataFrame[article_id: int, t_dat: date, customer_id: string, price: double, sales_channel_id: int, product_group_name: string, index_name: string, purchase_year: int, purchase_month: int]

In [31]:
#Define Seasons

In [32]:
trend_df = master_df.withColumn(
    "season",
    when(col("purchase_month").isin(lit(3), lit(4), lit(5)), lit("Spring"))
    .when(col("purchase_month").isin(lit(6), lit(7), lit(8)), lit("Summer"))
    .when(col("purchase_month").isin(lit(9), lit(10), lit(11)), lit("Autumn"))
    .otherwise(lit("Winter"))
)

In [34]:
#Aggregation total revenue by Season and Product Group

In [36]:
season_trend = trend_df.groupBy("season", "product_group_name")\
    .agg(sum("price").alias("total_revenue"))\
    .orderBy(col("season"), col("total_revenue").desc())

print("\n--- A. Product Trend Changes by Season (Top 50) ---")
season_trend.show(50, truncate=False)


--- A. Product Trend Changes by Season (Top 50) ---
+------+-------------------+--------------------+
|season|product_group_name |total_revenue       |
+------+-------------------+--------------------+
|Autumn|Garment Upper body |1227.381999999985   |
|Autumn|Garment Lower body |616.2010508474555   |
|Autumn|Garment Full body  |227.38094915254277  |
|Autumn|Underwear          |122.67283050847382  |
|Autumn|Shoes              |87.1528305084747    |
|Autumn|Accessories        |77.69566101694934   |
|Autumn|Nightwear          |27.74450847457622   |
|Autumn|Socks & Tights     |27.54561016949132   |
|Autumn|Swimwear           |26.142525423728685  |
|Autumn|Unknown            |1.2983728813559297  |
|Autumn|Bags               |0.3461186440677959  |
|Autumn|Items              |0.0863389830508472  |
|Autumn|Underwear/nightwear|0.0677627118644066  |
|Autumn|Cosmetic           |0.044305084745762294|
|Autumn|Interior textile   |0.0169322033898305  |
|Spring|Garment Upper body |796.6026440677904  

In [37]:
#Product trends show Garment Upper body is the consistent top revenue driver across all seasons, while Swimwear exhibits a pronounced seasonal shift, generating its highest sales during the Spring and Summer months.

In [39]:
#Aggregation total revenue by Year and Product Group

In [40]:
year_trend = trend_df.groupBy("purchase_year", "product_group_name")\
    .agg(sum("price").alias("total_revenue"))\
    .orderBy(col("purchase_year"), col("total_revenue").desc())

print("\n--- B. Product Trend Changes by Year (Top 50) ---")
year_trend.show(50, truncate=False)


--- B. Product Trend Changes by Year (Top 50) ---
+-------------+-------------------+-------------------+
|purchase_year|product_group_name |total_revenue      |
+-------------+-------------------+-------------------+
|2018         |Garment Upper body |1318.7904406779617 |
|2018         |Garment Lower body |646.2308135593173  |
|2018         |Garment Full body  |252.7640847457632  |
|2018         |Underwear          |141.2368644067784  |
|2018         |Shoes              |93.36316949152548  |
|2018         |Accessories        |91.40516949152514  |
|2018         |Socks & Tights     |35.029355932203245 |
|2018         |Nightwear          |34.77883050847453  |
|2018         |Swimwear           |29.278593220338866 |
|2018         |Unknown            |1.5160338983050816 |
|2018         |Bags               |0.10164406779660991|
|2018         |Cosmetic           |0.06883050847457561|
|2018         |Items              |0.0623220338983049 |
|2018         |Underwear/nightwear|0.0558983050847456

In [43]:
#2. Suggest a methodology and make an estimate for the whole data given that transactions.csv is a random sample of 3%
#from the entire population (3 points). Estimate
 #   a. total revenue
 #  b. number of customers
 # c. number of transactions
#d. average expenses per year per customer

In [44]:
from pyspark.sql.functions import sum, countDistinct, lit, count, to_date, year, col

In [45]:
# Define the scaling factor
EXTRAPOLATION_FACTOR = 1 / 0.03

In [46]:
#Calculate Sample Metrics
sample_metrics = transactions_df.agg(
    sum("price").alias("sample_total_revenue"),
    countDistinct("customer_id").alias("sample_total_customers"),
    count(lit(1)).alias("sample_total_transactions") # count(lit(1)) counts all rows
).collect()[0]

In [47]:
# Extract Sample Values
S_Revenue = sample_metrics["sample_total_revenue"]
S_Customers = sample_metrics["sample_total_customers"]
S_Transactions = sample_metrics["sample_total_transactions"]

In [48]:
#Determine the number of years in the sample for calculation average expenses per year per customer

In [49]:
dates_df = transactions_df.withColumn("t_dat", to_date(col("t_dat"), "yyyy-MM-dd"))

In [50]:
min_year_val = dates_df.select(year("t_dat")).agg({"year(t_dat)": "min"}).collect()[0][0]
max_year_val = dates_df.select(year("t_dat")).agg({"year(t_dat)": "max"}).collect()[0][0]

In [51]:
NUM_YEARS = max_year_val - min_year_val + 1
# Handle case where date range is minimal or calculation is zero
if NUM_YEARS == 0 or NUM_YEARS is None: NUM_YEARS = 1

In [53]:
#total revenue, number of customers,number of transactions are scaled

In [52]:
E_Revenue = S_Revenue * EXTRAPOLATION_FACTOR
E_Customers = S_Customers * EXTRAPOLATION_FACTOR
E_Transactions = S_Transactions * EXTRAPOLATION_FACTOR

In [55]:
#average expenses per year per customer is a sample average, NOT scaled


In [54]:
Avg_Expenses_Per_Year_Per_Customer = S_Revenue / (S_Customers * NUM_YEARS)

In [56]:
print("\n--- Estimated Population Metrics (3% Sample) ---")
print(f"Extrapolation Factor (F): {EXTRAPOLATION_FACTOR:.3f}")
print("-" * 60)
print(f"a. Estimated Total Revenue: ${E_Revenue:,.2f}")
print(f"b. Estimated Number of Customers: {E_Customers:,.0f}")
print(f"c. Estimated Number of Transactions: {E_Transactions:,.0f}")
print("-" * 60)
print(f"d. Average Expenses per Year Per Customer (Sample Average): ${Avg_Expenses_Per_Year_Per_Customer:.4f}")


--- Estimated Population Metrics (3% Sample) ---
Extrapolation Factor (F): 33.333
------------------------------------------------------------
a. Estimated Total Revenue: $308,054.91
b. Estimated Number of Customers: 7,689,600
c. Estimated Number of Transactions: 11,202,600
------------------------------------------------------------
d. Average Expenses per Year Per Customer (Sample Average): $0.0200


In [58]:
#Final conclusion
#The company is estimated to serve nearly 7.7$ million customers.This massive number means the company has a huge market reach.
#The company sells a huge amount of stuff, with over $11$ million estimated transactions.
#All these transactions generate an estimated Total Revenue of over 308,000$.
#The average expense per customer per year is estimated at a very low.
#we know the prices are normalized, this low number means that, on average, customers are making many small-value purchases.

In [59]:
#Extra 2 points for more findings and creative work
#This part i will do after your feadback.